In [48]:
# Dependencies
import pandas as pd
import requests
import json
import time
import sqlite3 as db

# Ignore SQLITE warnings related to Decimal numbers in the Chinook database.
import warnings
warnings.filterwarnings('ignore')

# Import Dependencies for SQLALCHEMY
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import create_engine, ForeignKey, MetaData, Table, select
from sqlalchemy import Column, Date, Integer, String, Numeric, Float

from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base

# Create an engine for the sqlite database
engine = create_engine("sqlite:///data/climate_change.sqlite", echo=False)
conn = engine.connect()


# EIA developer API key
from config import api_key

# Build the endpoint URL
# http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=EMISS.CO2-TOTV-TT-PE-MD.A
#target_url = 'http://api.eia.gov/category/?api_key=3f7c6e0096672dc16b49796f3a89c943&category_id=240690'
url='http://api.eia.gov/category/?'
id='2251670'
query_url = url + "api_key=" + api_key + "&category_id=" + id

In [49]:
# Run a request to endpoint and convert result to json
co2_data = requests.get(query_url).json()

# Print the json
print(json.dumps(co2_data, indent=4, sort_keys=True))

{
    "category": {
        "category_id": "2251670",
        "childcategories": [
            {
                "category_id": 2251671,
                "name": "Maryland"
            },
            {
                "category_id": 2251672,
                "name": "Alabama"
            },
            {
                "category_id": 2251673,
                "name": "Alaska"
            },
            {
                "category_id": 2251674,
                "name": "Arizona"
            },
            {
                "category_id": 2251675,
                "name": "Arkansas"
            },
            {
                "category_id": 2251676,
                "name": "California"
            },
            {
                "category_id": 2251677,
                "name": "Colorado"
            },
            {
                "category_id": 2251678,
                "name": "Connecticut"
            },
            {
                "category_id": 2251679,
                "name": "Delaw

In [50]:
# Count No of state and get the chield category_id
state_count=(co2_data["category"]["childcategories"])
print (len(state_count))
print(co2_data["category"]["childcategories"][0]['category_id'])

52
2251671


In [51]:
# Declare a place holder array to append category_id of each state 
state_id=[]
i=0
while i < len(state_count) :
    state_id.append(co2_data["category"]["childcategories"][i]['category_id'])
    i+=1

In [52]:
state_id[44]

2251715

In [53]:
len(state_id)

52

In [54]:
# Declare a place holder array to append serise_id of each state(Range 0 to 43 and 45 to end of array)
# Note: The 44th category id 2251715 ("name": "United States") has unknown error and excluded intentionally 
           
serise_id=[]

url='http://api.eia.gov/category/?'
#id="2251671"
i=0
#j=0
while i < 44 :    
    id=str(state_id[i])
    query_url = url + "api_key=" + api_key + "&category_id=" + id    
    co2_data = requests.get(query_url).json()    
    serise_id.append(co2_data["category"]["childseries"][59]["series_id"])   
    i+=1

In [55]:
i=45
while i < len(state_id) :    
    id=str(state_id[i])
    query_url = url + "api_key=" + api_key + "&category_id=" + id    
    co2_data = requests.get(query_url).json()    
    serise_id.append(co2_data["category"]["childseries"][59]["series_id"])    
    i+=1

In [56]:
serise_id

['EMISS.CO2-TOTV-TT-TO-MD.A',
 'EMISS.CO2-TOTV-TT-TO-AL.A',
 'EMISS.CO2-TOTV-TT-TO-AK.A',
 'EMISS.CO2-TOTV-TT-TO-AZ.A',
 'EMISS.CO2-TOTV-TT-TO-AR.A',
 'EMISS.CO2-TOTV-TT-TO-CA.A',
 'EMISS.CO2-TOTV-TT-TO-CO.A',
 'EMISS.CO2-TOTV-TT-TO-CT.A',
 'EMISS.CO2-TOTV-TT-TO-DE.A',
 'EMISS.CO2-TOTV-TT-TO-DC.A',
 'EMISS.CO2-TOTV-TT-TO-FL.A',
 'EMISS.CO2-TOTV-TT-TO-GA.A',
 'EMISS.CO2-TOTV-TT-TO-HI.A',
 'EMISS.CO2-TOTV-TT-TO-ID.A',
 'EMISS.CO2-TOTV-TT-TO-IL.A',
 'EMISS.CO2-TOTV-TT-TO-IN.A',
 'EMISS.CO2-TOTV-TT-TO-IA.A',
 'EMISS.CO2-TOTV-TT-TO-KS.A',
 'EMISS.CO2-TOTV-TT-TO-KY.A',
 'EMISS.CO2-TOTV-TT-TO-LA.A',
 'EMISS.CO2-TOTV-TT-TO-ME.A',
 'EMISS.CO2-TOTV-TT-TO-MA.A',
 'EMISS.CO2-TOTV-TT-TO-MI.A',
 'EMISS.CO2-TOTV-TT-TO-MN.A',
 'EMISS.CO2-TOTV-TT-TO-MS.A',
 'EMISS.CO2-TOTV-TT-TO-MO.A',
 'EMISS.CO2-TOTV-TT-TO-MT.A',
 'EMISS.CO2-TOTV-TT-TO-NE.A',
 'EMISS.CO2-TOTV-TT-TO-NV.A',
 'EMISS.CO2-TOTV-TT-TO-NH.A',
 'EMISS.CO2-TOTV-TT-TO-NJ.A',
 'EMISS.CO2-TOTV-TT-TO-NM.A',
 'EMISS.CO2-TOTV-TT-TO-NY.A',
 'EMISS.CO

In [57]:
len(serise_id)

51

In [58]:
serise_id[6]

'EMISS.CO2-TOTV-TT-TO-CO.A'

In [59]:
x=serise_id[6]
ab=x.split('-',1)[1]
ab=ab.split('-',1)[1]
ab=ab.split('-',1)[1]
ab=ab.split('-',1)[1]
ab=ab.split('.',1)[0]
ab

'CO'

In [60]:
# Create an Empty DataFrame with col Year CO2DATA and State 
df=pd.DataFrame(columns=['Abbr','State','Year', 'CO2DATA' ])
# Create API call and get data and accumulate on df
url='https://api.eia.gov/series/?'
for x in serise_id:      
    sid=x
    # Create query url with serise id
    query_url = url + "api_key=" + api_key + "&series_id=" + sid

    # Run a request to endpoint and convert result to json
    co2_data = requests.get(query_url).json()
    # Get the State Abbr
    ab=x.split('-',1)[1]
    ab=ab.split('-',1)[1]
    ab=ab.split('-',1)[1]
    ab=ab.split('-',1)[1]
    ab=ab.split('.',1)[0]


    # Get the state name
    stname=co2_data["series"][0]['name']    
    nstate=stname.split(',',1)[1]
    stateName=nstate.split(',',1)[1].lstrip() 
    ydata=co2_data["series"][0]['data']
    # Create new df with data 
    co2df=pd.DataFrame(ydata,columns=['Year','CO2DATA'])
    # Add state name as column to df
    co2df['State']=stateName
    # Add state Abbr as column to df
    co2df['Abbr']=ab
    # Append df
    df=df.append(co2df)

In [61]:
#df=df.reset_index(drop=True)
#df.to_csv('all_state_co2.csv')
#df

In [62]:
df.head()

,Abbr,CO2DATA,State,Year
0,MD,57.557961,Maryland,2016
1,MD,59.473748,Maryland,2015
2,MD,61.391691,Maryland,2014
3,MD,59.152914,Maryland,2013
4,MD,59.859472,Maryland,2012


In [63]:
# Create a database with table model or schema
Base = declarative_base()

class co2(Base):

    __tablename__ = "co2"

    ID = Column(Integer, primary_key=True) 
    Abbr=Column(String)
    State = Column(String) 
    Year = Column(Integer)
    CO2DATA = Column(Float)
    

    def __init__(self, name):
        self.name = name  

Base.metadata.create_all(engine)

In [64]:
# Query all database and make sure the new one created
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
print(Base.classes.keys())
co2 = Base.classes.co2

['co', 'co2']


In [65]:
# Create a database session object
session = Session(engine)
tableToWriteTo = 'co2'

In [66]:
listToWrite = df.to_dict(orient='records')
listToWrite

[{'Abbr': 'MD', 'CO2DATA': 57.557961, 'State': 'Maryland', 'Year': '2016'},
 {'Abbr': 'MD', 'CO2DATA': 59.473748, 'State': 'Maryland', 'Year': '2015'},
 {'Abbr': 'MD', 'CO2DATA': 61.391691, 'State': 'Maryland', 'Year': '2014'},
 {'Abbr': 'MD', 'CO2DATA': 59.152914, 'State': 'Maryland', 'Year': '2013'},
 {'Abbr': 'MD', 'CO2DATA': 59.859472, 'State': 'Maryland', 'Year': '2012'},
 {'Abbr': 'MD', 'CO2DATA': 64.432162, 'State': 'Maryland', 'Year': '2011'},
 {'Abbr': 'MD', 'CO2DATA': 69.124429, 'State': 'Maryland', 'Year': '2010'},
 {'Abbr': 'MD', 'CO2DATA': 70.482085, 'State': 'Maryland', 'Year': '2009'},
 {'Abbr': 'MD', 'CO2DATA': 73.767925, 'State': 'Maryland', 'Year': '2008'},
 {'Abbr': 'MD', 'CO2DATA': 77.533365, 'State': 'Maryland', 'Year': '2007'},
 {'Abbr': 'MD', 'CO2DATA': 77.166737, 'State': 'Maryland', 'Year': '2006'},
 {'Abbr': 'MD', 'CO2DATA': 82.005117, 'State': 'Maryland', 'Year': '2005'},
 {'Abbr': 'MD', 'CO2DATA': 79.851084, 'State': 'Maryland', 'Year': '2004'},
 {'Abbr': 'M

In [67]:
metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

In [68]:
conn.execute(table.insert(), listToWrite)

In [69]:
session.commit()

In [70]:
# display all tables
Base.classes.keys()

['co', 'co2']

In [78]:
co2 = Base.classes.co2

In [79]:
t = Table("co2", metadata, autoload=True)
columns = [m.key for m in t.columns]
columns

['ID', 'Abbr', 'State', 'Year', 'CO2DATA']

In [80]:
# List all of the data found in the invoices table 
session.query(co2.ID,co2.Abbr,co2.State, co2.Year, co2.CO2DATA).all()

[(1, 'MD', 'Maryland', 2016, 57.557961),
 (2, 'MD', 'Maryland', 2015, 59.473748),
 (3, 'MD', 'Maryland', 2014, 61.391691),
 (4, 'MD', 'Maryland', 2013, 59.152914),
 (5, 'MD', 'Maryland', 2012, 59.859472),
 (6, 'MD', 'Maryland', 2011, 64.432162),
 (7, 'MD', 'Maryland', 2010, 69.124429),
 (8, 'MD', 'Maryland', 2009, 70.482085),
 (9, 'MD', 'Maryland', 2008, 73.767925),
 (10, 'MD', 'Maryland', 2007, 77.533365),
 (11, 'MD', 'Maryland', 2006, 77.166737),
 (12, 'MD', 'Maryland', 2005, 82.005117),
 (13, 'MD', 'Maryland', 2004, 79.851084),
 (14, 'MD', 'Maryland', 2003, 79.299126),
 (15, 'MD', 'Maryland', 2002, 76.48787),
 (16, 'MD', 'Maryland', 2001, 76.532899),
 (17, 'MD', 'Maryland', 2000, 76.780629),
 (18, 'MD', 'Maryland', 1999, 76.783697),
 (19, 'MD', 'Maryland', 1998, 74.368834),
 (20, 'MD', 'Maryland', 1997, 71.775523),
 (21, 'MD', 'Maryland', 1996, 71.377163),
 (22, 'MD', 'Maryland', 1995, 69.305464),
 (23, 'MD', 'Maryland', 1994, 69.840096),
 (24, 'MD', 'Maryland', 1993, 68.832021),
 (